In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import os.path
from bs4 import BeautifulSoup
import requests

In [36]:
first_record = 20000
last_record = 22317 + 1
record_segment_file_name = '../../../2_data/' + 'record_segment_good_length_' + str(first_record) + '_' + str(last_record - 1) +'.csv'

In [37]:
record_segment_file_name

'../../../2_data/record_segment_good_length_20000_22317.csv'

In [2]:
record_path_local = '../../../2_data/record_path_mimic3_matched.csv'
record_root_url = 'https://physionet.org/content/mimic3wdb/1.0/matched'
record_path_url = record_root_url + '/RECORDS-waveforms'

signal_extract_name = 'PLETH'
signal_extract_minimum_length = 5 # the minimum length of the signal segment, in minutes

#record_root_url = 'https://archive.physionet.org/physiobank/database/mimic3wdb'
#record_path_url = record_root_url + '/RECORDS'

print('record_path_local: {}'.format(record_path_local))
print('record_root_url: {}'.format(record_root_url))
print('record_path_url: {}'.format(record_path_url))

record_path_local: ../../../2_data/record_path_mimic3_matched.csv
record_root_url: https://physionet.org/content/mimic3wdb/1.0/matched
record_path_url: https://physionet.org/content/mimic3wdb/1.0/matched/RECORDS-waveforms


In [4]:
def get_url_content(url, tag=None):
    
    '''
    This function scrapes a list of useful information from a given PhysioNet URL.
    If the URL address points to an HTML document, the information to be extracted is define by a tag.
    I found this address (https://hackersandslackers.com/scraping-urls-with-beautifulsoup/) usefull
      for details on how to scrape a web page
    '''
    
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }    
    
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    if '<!DOCTYPE html>' in str(soup):
        content = getattr(soup, str(tag)).getText()
        #content = soup.pre.getText()
    else:
        content = soup.getText()
    return content.splitlines()

In [5]:
tag = 'pre'
record_path_list = get_url_content(record_path_url, tag)
print(len(record_path_list))
for i in range(5):
    print('record_path: {}'.format(record_path_list[i]))

22317
record_path: p00/p000020/p000020-2183-04-28-17-47
record_path: p00/p000030/p000030-2172-10-16-12-22
record_path: p00/p000033/p000033-2116-12-24-12-35
record_path: p00/p000033/p000033-2116-12-25-13-11
record_path: p00/p000052/p000052-2191-01-10-02-21


In [40]:
tag = 'pre'
last_record = len(record_path_list) + 1
record_good_length_index = 0
for process_record_index in range(first_record, last_record):
        
    #print(process_record_index)
    record_path = record_path_list[process_record_index]
    print('processing record #: {}, record_path: {}'.format(process_record_index, record_path))
    
    #directory p04/p044083.
    directory = str(record_path.split("/")[0]) + '/' + str(record_path.split("/")[1])
    #print('directory: {}'.format(directory))
    
    intermediate_directory = str(record_path.split("/")[0])
    #print('intermediate_directory: {}'.format(intermediate_directory))
    subject_id = record_path.split("/")[1]
    subject_id = int(subject_id[1:])
    #print('subject_id: {}'.format(subject_id))
        
    master_waveform_header = record_root_url + '/' + record_path + '.hea'
    #print('master_waveform_header: {}'.format(master_waveform_header))
    
    target_url = master_waveform_header
    master_waveform_content = get_url_content(target_url, tag)
    #print('master_waveform_content: {}'.format(master_waveform_content))
    
    #print('       ...........\n')
    
    master_waveform_content_first_line = master_waveform_content[0].split(' ')
    record_length = int(master_waveform_content_first_line[3]) / int(master_waveform_content_first_line[2])
    #print('record_length (s): {}'.format(record_length))
    
    #print('       ...........\n')    
        
    waveform_layout_header = master_waveform_content[1].split(' ')[0]
    waveform_layout_header = waveform_layout_header + '.hea'
    waveform_layout_header = record_root_url + '/' + directory + '/' + waveform_layout_header
    #print('waveform_layout_header: {}'.format(waveform_layout_header))    
    target_url = waveform_layout_header
    waveform_layout_content = get_url_content(target_url, tag)
    #print('waveform_layout_content: {}'.format(waveform_layout_content))
        
    signal_names = [x.split(' ')[-1] for x in waveform_layout_content[1:-1]]
    #print('signal_names: {}'.format(signal_names))
    
    if signal_extract_name in signal_names and record_length > 60 * signal_extract_minimum_length: # and len(record_segments_path_list):      
        
        record_good_length_index += 1
        
        # Open the record_segment_file_name file with access mode 'a'
        record_segment_file_object = open(record_segment_file_name, 'a')

        # get the waveform record name
        waveform_record_name = [x.split(' ')[0].split('_') for x in master_waveform_content if '3' == x[0]][0][0]
        
        # get the waveform_record_state
        waveform_record_state = [(waveform_record_name + '_' in x) and not (waveform_record_name + '_layout' in x) 
                for x in master_waveform_content]

        # get the waveform_record_segment
        waveform_record_segment = [master_waveform_content[x].split(' ') for x, y in enumerate(waveform_record_state) if y]

        
        for segment_index in range(len(waveform_record_segment)):
            # number of samples: int(waveform_record_segment[segment_index][1])
            # sampling frequency: int(master_waveform_content_first_line[2])
            # length (in seconds) of the record segment: number of samples / sampling frequency
            
            if int(waveform_record_segment[segment_index][1]) / int(master_waveform_content_first_line[2]) > 60 * signal_extract_minimum_length:
                
                waveform_record_segment_good_length_root = directory + '/' +  waveform_record_segment[segment_index][0]
                print('  ' + str(record_good_length_index) + ',' + str(process_record_index) + ',' + waveform_record_segment_good_length_root)
                
                # Append the waveform_record_segment_good_length_root at the end of file record_segment_file_name
                record_segment_file_object.write(str(record_good_length_index) + ',' + str(process_record_index) + ',' + waveform_record_segment_good_length_root + '\n')

        #print('baubau_1')
        
        # Close the file record_segment_file_name
        record_segment_file_object.close()
        #break
        
    
    
    #print('       ...........\n')
    #index_of_last_record_processed = index_of_last_record_processed + 1
    print('  \nindex of last record processed successfully: {}'.format(process_record_index))  
    print('--------------------------------------')

    
    


processing record #: 20000, record_path: p09/p090354/p090354-2146-09-13-17-27
  
index of last record processed successfully: 20000
--------------------------------------
processing record #: 20001, record_path: p09/p090354/p090354-2146-09-18-01-38
  
index of last record processed successfully: 20001
--------------------------------------
processing record #: 20002, record_path: p09/p090354/p090354-2146-09-23-16-46
  
index of last record processed successfully: 20002
--------------------------------------
processing record #: 20003, record_path: p09/p090354/p090354-2146-09-25-16-33
  1,20003,p09/p090354/3422519_0002
  1,20003,p09/p090354/3422519_0005
  1,20003,p09/p090354/3422519_0006
  1,20003,p09/p090354/3422519_0008
  1,20003,p09/p090354/3422519_0010
  1,20003,p09/p090354/3422519_0014
  1,20003,p09/p090354/3422519_0015
  1,20003,p09/p090354/3422519_0017
  1,20003,p09/p090354/3422519_0020
  1,20003,p09/p090354/3422519_0024
  1,20003,p09/p090354/3422519_0028
  1,20003,p09/p090354/34

  
index of last record processed successfully: 20024
--------------------------------------
processing record #: 20025, record_path: p09/p090410/p090410-2103-06-25-15-46
  10,20025,p09/p090410/3010012_0007
  10,20025,p09/p090410/3010012_0008
  10,20025,p09/p090410/3010012_0011
  10,20025,p09/p090410/3010012_0012
  10,20025,p09/p090410/3010012_0014
  10,20025,p09/p090410/3010012_0018
  10,20025,p09/p090410/3010012_0019
  10,20025,p09/p090410/3010012_0022
  10,20025,p09/p090410/3010012_0025
  10,20025,p09/p090410/3010012_0027
  10,20025,p09/p090410/3010012_0029
  10,20025,p09/p090410/3010012_0031
  10,20025,p09/p090410/3010012_0033
  10,20025,p09/p090410/3010012_0035
  10,20025,p09/p090410/3010012_0043
  10,20025,p09/p090410/3010012_0049
  10,20025,p09/p090410/3010012_0051
  10,20025,p09/p090410/3010012_0056
  10,20025,p09/p090410/3010012_0059
  10,20025,p09/p090410/3010012_0062
  10,20025,p09/p090410/3010012_0064
  10,20025,p09/p090410/3010012_0066
  10,20025,p09/p090410/3010012_0068
 

  
index of last record processed successfully: 20052
--------------------------------------
processing record #: 20053, record_path: p09/p090484/p090484-2133-05-12-15-30
  
index of last record processed successfully: 20053
--------------------------------------
processing record #: 20054, record_path: p09/p090493/p090493-2192-06-04-23-10
  
index of last record processed successfully: 20054
--------------------------------------
processing record #: 20055, record_path: p09/p090495/p090495-2148-03-28-17-49
  
index of last record processed successfully: 20055
--------------------------------------
processing record #: 20056, record_path: p09/p090495/p090495-2148-03-29-11-19
  
index of last record processed successfully: 20056
--------------------------------------
processing record #: 20057, record_path: p09/p090522/p090522-2107-08-08-18-00
  
index of last record processed successfully: 20057
--------------------------------------
processing record #: 20058, record_path: p09/p090522

  
index of last record processed successfully: 20061
--------------------------------------
processing record #: 20062, record_path: p09/p090533/p090533-2178-12-15-02-54
  22,20062,p09/p090533/3950441_0001
  22,20062,p09/p090533/3950441_0002
  22,20062,p09/p090533/3950441_0004
  22,20062,p09/p090533/3950441_0007
  22,20062,p09/p090533/3950441_0008
  22,20062,p09/p090533/3950441_0010
  22,20062,p09/p090533/3950441_0012
  22,20062,p09/p090533/3950441_0015
  22,20062,p09/p090533/3950441_0016
  22,20062,p09/p090533/3950441_0020
  22,20062,p09/p090533/3950441_0023
  22,20062,p09/p090533/3950441_0026
  22,20062,p09/p090533/3950441_0028
  22,20062,p09/p090533/3950441_0030
  22,20062,p09/p090533/3950441_0032
  22,20062,p09/p090533/3950441_0035
  22,20062,p09/p090533/3950441_0037
  22,20062,p09/p090533/3950441_0038
  22,20062,p09/p090533/3950441_0041
  22,20062,p09/p090533/3950441_0043
  22,20062,p09/p090533/3950441_0046
  22,20062,p09/p090533/3950441_0047
  22,20062,p09/p090533/3950441_0048
 

  
index of last record processed successfully: 20082
--------------------------------------
processing record #: 20083, record_path: p09/p090629/p090629-2136-10-23-17-23
  
index of last record processed successfully: 20083
--------------------------------------
processing record #: 20084, record_path: p09/p090648/p090648-2197-06-10-18-29
  33,20084,p09/p090648/3414881_0003
  33,20084,p09/p090648/3414881_0005
  33,20084,p09/p090648/3414881_0007
  33,20084,p09/p090648/3414881_0008
  33,20084,p09/p090648/3414881_0012
  33,20084,p09/p090648/3414881_0016
  33,20084,p09/p090648/3414881_0023
  33,20084,p09/p090648/3414881_0028
  33,20084,p09/p090648/3414881_0032
  33,20084,p09/p090648/3414881_0036
  33,20084,p09/p090648/3414881_0038
  33,20084,p09/p090648/3414881_0040
  33,20084,p09/p090648/3414881_0042
  
index of last record processed successfully: 20084
--------------------------------------
processing record #: 20085, record_path: p09/p090649/p090649-2198-12-16-18-23
  34,20085,p09/p090

...

  
index of last record processed successfully: 22235
--------------------------------------
processing record #: 22236, record_path: p09/p099715/p099715-2150-11-06-09-51
  
index of last record processed successfully: 22236
--------------------------------------
processing record #: 22237, record_path: p09/p099715/p099715-2150-11-07-09-53
  
index of last record processed successfully: 22237
--------------------------------------
processing record #: 22238, record_path: p09/p099740/p099740-2135-05-09-05-08
  
index of last record processed successfully: 22238
--------------------------------------
processing record #: 22239, record_path: p09/p099740/p099740-2135-05-09-16-16
  
index of last record processed successfully: 22239
--------------------------------------
processing record #: 22240, record_path: p09/p099740/p099740-2135-05-09-20-37
  
index of last record processed successfully: 22240
--------------------------------------
processing record #: 22241, record_path: p09/p099740

  968,22267,p09/p099785/3995417_0002
  968,22267,p09/p099785/3995417_0004
  968,22267,p09/p099785/3995417_0011
  968,22267,p09/p099785/3995417_0012
  968,22267,p09/p099785/3995417_0014
  968,22267,p09/p099785/3995417_0016
  968,22267,p09/p099785/3995417_0017
  968,22267,p09/p099785/3995417_0018
  968,22267,p09/p099785/3995417_0020
  968,22267,p09/p099785/3995417_0026
  968,22267,p09/p099785/3995417_0028
  968,22267,p09/p099785/3995417_0034
  968,22267,p09/p099785/3995417_0036
  968,22267,p09/p099785/3995417_0038
  968,22267,p09/p099785/3995417_0040
  968,22267,p09/p099785/3995417_0042
  968,22267,p09/p099785/3995417_0044
  968,22267,p09/p099785/3995417_0046
  968,22267,p09/p099785/3995417_0048
  968,22267,p09/p099785/3995417_0049
  968,22267,p09/p099785/3995417_0050
  968,22267,p09/p099785/3995417_0051
  968,22267,p09/p099785/3995417_0052
  968,22267,p09/p099785/3995417_0053
  968,22267,p09/p099785/3995417_0057
  968,22267,p09/p099785/3995417_0058
  968,22267,p09/p099785/3995417_0060
 

  972,22283,p09/p099836/3585140_0001
  972,22283,p09/p099836/3585140_0004
  972,22283,p09/p099836/3585140_0007
  972,22283,p09/p099836/3585140_0009
  972,22283,p09/p099836/3585140_0011
  972,22283,p09/p099836/3585140_0014
  972,22283,p09/p099836/3585140_0017
  972,22283,p09/p099836/3585140_0019
  972,22283,p09/p099836/3585140_0021
  972,22283,p09/p099836/3585140_0024
  972,22283,p09/p099836/3585140_0027
  
index of last record processed successfully: 22283
--------------------------------------
processing record #: 22284, record_path: p09/p099863/p099863-2142-04-02-20-38
  973,22284,p09/p099863/3650177_0005
  973,22284,p09/p099863/3650177_0006
  973,22284,p09/p099863/3650177_0007
  973,22284,p09/p099863/3650177_0008
  973,22284,p09/p099863/3650177_0017
  973,22284,p09/p099863/3650177_0021
  973,22284,p09/p099863/3650177_0023
  973,22284,p09/p099863/3650177_0027
  973,22284,p09/p099863/3650177_0030
  
index of last record processed successfully: 22284
-----------------------------------

  
index of last record processed successfully: 22301
--------------------------------------
processing record #: 22302, record_path: p09/p099913/p099913-2141-05-12-13-30
  
index of last record processed successfully: 22302
--------------------------------------
processing record #: 22303, record_path: p09/p099922/p099922-2107-04-07-12-17
  
index of last record processed successfully: 22303
--------------------------------------
processing record #: 22304, record_path: p09/p099946/p099946-2161-07-14-21-47
  981,22304,p09/p099946/3382559_0008
  981,22304,p09/p099946/3382559_0010
  981,22304,p09/p099946/3382559_0012
  981,22304,p09/p099946/3382559_0015
  981,22304,p09/p099946/3382559_0017
  
index of last record processed successfully: 22304
--------------------------------------
processing record #: 22305, record_path: p09/p099955/p099955-2139-07-14-04-45
  
index of last record processed successfully: 22305
--------------------------------------
processing record #: 22306, record_pat

IndexError: list index out of range

In [ ]:
gigi = 2

In [ ]:
gigi +=1

In [ ]:
gigi